# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [12]:
import copy
import json
import random
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate
from config import PALM_CONFIG, GEMINI_CONFIG, CLAUDE_CONFIG

In [13]:
seed = 420
now = datetime.now().strftime("%Y%m%d")

In [14]:
dataset = load_dataset("cais/mmlu", "all")

# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(100))

# # user provided data description
# DESCRIPTION = """
# This is a massive multitask test consisting of multiple-choice questions from various branches of knowledge.
# The test spans subjects in the humanities, social sciences, hard sciences, and other areas that are important for some people to learn.
# To attain high accuracy on this test, models must possess extensive world knowledge and problem solving ability.
# This covers 57 subjects  across STEM, the humanities, the social sciences, and more. 
# It ranges in difficulty from an elementary level to an advanced professional level, and it tests both world knowledge and problem solving ability. 
# Subjects range from traditional areas, such as mathematics and history, to more specialized areas like law and ethics.
# """


In [ ]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [ ]:
prompt = [gemini_prompt_template.format(datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

In [ ]:
models = [
    "palm",
    "gemini",
    # "claude"
    ]

PALM_CONFIG["project_config"]["qpm"] = 150

palm_1 =  copy.deepcopy(PALM_CONFIG)
palm_1['config_name'] = "temp_0.4"

palm_2 =  copy.deepcopy(PALM_CONFIG)
palm_2['config_name'] = "temp_0.9"
palm_2["generation_config"]['temperature'] = 0.9

GEMINI_CONFIG["project_config"]["qpm"] = 100

gemini_1 =  copy.deepcopy(GEMINI_CONFIG)
gemini_1['config_name'] = "-1.0-pro-002"

gemini_2 =  copy.deepcopy(GEMINI_CONFIG)
gemini_2['config_name'] = "-1.5-flash-001"
gemini_2['"model"'] = "gemini-1.5-flash-001"

gemini_3 =  copy.deepcopy(GEMINI_CONFIG)
gemini_3['config_name'] = "-1.0-ultra-001"
gemini_3['"model"'] = "gemini-1.0-ultra-001"



model_config = {
    "gemini": [
        gemini_1,
        gemini_2,
        gemini_3
         ],
    "palm": [
        palm_1, 
        palm_2
        ],
    # "claude": [
    #     CLAUDE_CONFIG
    # ]
}

ann = Annotate()


In [ ]:
output_dict = await ann.classification(prompt, models, model_config)

In [ ]:
llm_response = {}

for k in output_dict.keys():
    llm_response[k] = []
    for idx, r in enumerate(output_dict[k]):
        if r is not None:
            stripped_r = r.strip().strip("'")
            if stripped_r in dataset['choices'][idx]:
                llm_response[k].append(dataset['choices'][idx].index(stripped_r))
            else:
                # Handle case where stripped_r is not found in choices
                llm_response[k].append(None)
        else:
            # Handle None values appropriately
            llm_response[k].append(None)

In [ ]:
with open(f"./data/raw_llm_response__{now}.json", "w") as json_file:
    json.dump(llm_response, json_file)

In [ ]:
def replace_none_with_random(data, n):
    new_data = {}  
    for key, lst in data.items():
        new_lst = []  
        for value in lst:
            if value is None:
                new_lst.append(random.randint(0, n))
            else:
                new_lst.append(value)
        new_data[key] = new_lst 
    return new_data



def convert_dict_to_indexed_list(data_dict):
    number_map = {key: index for index, key in enumerate(data_dict.keys())}
    max_len = len(next(iter(data_dict.values())))

    result = []
    for index in range(max_len):
        for key, value_list in data_dict.items():
            value = value_list[index]
            # Convert to 0 if not an integer
            converted_value = 0 if not isinstance(value, int) else value 
            result.append([index, number_map[key], converted_value])
    return result
    

def generate_task_config(response_dict, num_classes):

    num_labels = sum(len(lst) for lst in response_dict.values())
    num_tasks =  len(list(response_dict.values())[0])
    num_labelers = len(response_dict)
    z  = 1/num_classes


    tc = [num_labels, num_labelers, num_tasks, num_classes]
    tc.extend([z] * tc[-1])

    return tc

In [ ]:
with open(f"./data/raw_llm_response__{now}.json", "w") as json_file:
    json.dump(llm_response, json_file)

In [ ]:
n_class = 4
llm_response_filled = replace_none_with_random(llm_response, n_class)


task_conf = generate_task_config(llm_response_filled, n_class)
llm_result_list = convert_dict_to_indexed_list(llm_response_filled)
llm_result_list.insert(0, task_conf)

In [ ]:
filename = f"./data/llm_response_100__{now}.txt"

with open(filename, "w") as file:
    for sublist in llm_result_list:
        line = " ".join(str(num) for num in sublist)  # Convert to string, join with spaces
        file.write(line + "\n")  # Write line and add newline
filename

#  GLAD

In [ ]:
from utils import glad

In [ ]:
glad_output = glad(filename)

# Eval

In [1]:
import json
import pandas as pd
from collections import Counter
from typing import Dict, List
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def get_majority_vote(label_dict: Dict[str, List[int]]) -> List[int]:
    """
    Finds the majority value for each element across multiple lists within a dictionary.

    Args:
        label_dict: A dictionary where keys are identifiers and values are lists of labels.

    Returns:
        A list of majority values corresponding to each element position.
    """
    list_of_labels = list(label_dict.values())  # Extract values into a list
    majority_values = []

    for elements in zip(*list_of_labels):
        element_counts = Counter(elements)
        most_common_element = element_counts.most_common(1)[0]
        majority_values.append(most_common_element[0])

    return majority_values


def accuracy_with_none_penalty(y_true, y_pred):
    filtered_y_true = []
    filtered_y_pred = []

    for true, pred in zip(y_true, y_pred):
        if pred is not None:  # Only include non-None predictions
            filtered_y_true.append(true)
            filtered_y_pred.append(pred)
        else:
            filtered_y_true.append(true)  # Include true label
            filtered_y_pred.append(-1)   # Replace None with wrong label (e.g., -1)

    return accuracy_score(filtered_y_true, filtered_y_pred)

In [8]:
with open("./data/raw_llm_response_100__20240529.json", "r") as f:
    llm_response = json.load(f)

df_glad = pd.read_csv("./data/label_glad__20240529.csv")


array([2, 0, 1, 3, 2, 1, 1, 2, 2, 2, 1, 2, 0, 2, 1, 1, 0, 1, 2, 1, 3, 2,
       1, 2, 2, 1, 1, 1, 2, 1, 3, 2, 0, 1, 1, 1, 0, 2, 2, 0, 3, 2, 1, 2,
       3, 3, 1, 1, 3, 3, 2, 2, 2, 3, 1, 1, 0, 2, 3, 2, 3, 3, 1, 2, 2, 3,
       3, 2, 3, 1, 1, 1, 3, 2, 1, 3, 1, 2, 1, 1, 2, 2, 0, 1, 0, 2, 2, 0,
       0, 1, 2, 0, 1, 0, 2, 0, 1, 2, 1, 3])

In [9]:
llm_response["majority"] = get_majority_vote(llm_response)
llm_response["glad"] = df_glad["label"].values

In [15]:
for k, v in llm_response.items():
    print(k, accuracy_with_none_penalty(dataset['answer'], v))

palm_temp_0.4 0.58
palm_temp_0.9 0.58
gemini_-1.0-pro-002 0.55
gemini_-1.5-flash-001 0.55
gemini_-1.0-ultra-001 0.53
majority 0.52
glad 0.6


In [ ]:

return {"accuracy": accuracy_score(y_true, y_pred),
                "f1_weighted": f1_score(y_true, y_pred, average='weighted'),
                "confusion_matrix": confusion_matrix(y_true, y_pred)
                }